In [1]:
import importlib

In [28]:
import hnsw
importlib.reload(hnsw)
from hnsw import HNSW
from hnsw import heuristic
import numpy as np
from datasets import load_sift_dataset, calculate_recall

In [18]:
import merge_hnsw
importlib.reload(merge_hnsw)
from merge_hnsw import hnsw_general_merge
from merge_hnsw import layer_merge2_func

In [2]:
distance_count = 0
def l2_distance(a, b):
    global distance_count
    distance_count+=1
    return np.linalg.norm(a - b)

In [4]:
hnsw_a = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)
hnsw_b = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)

In [5]:
hnsw_a.load('save/sift1m/hnsw_a.txt')
hnsw_b.load('save/sift1m/hnsw_b.txt')

In [6]:
merged_data = hnsw_a.data.copy()
merged_data.update(hnsw_b.data)

In [19]:
%%time
distance_count = 0
hnsw_merged2 = hnsw_general_merge(hnsw_a, hnsw_b, merged_data, layer_merge2_func)

Merging level: 0


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [58:56<00:00, 282.74it/s]


Merging level: 1


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62993/62993 [02:01<00:00, 518.32it/s]


Merging level: 2


...

Merging level: 3


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 272/272 [00:00<00:00, 986.60it/s]


Merging level: 4


Merging progress: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1774.73it/s]

CPU times: user 59min 24s, sys: 8min 26s, total: 1h 7min 50s
Wall time: 1h 1min 4s


In [25]:
hnsw_merged2.save('save/sift1m/hnsw_merged2.txt')

In [20]:
print(f'Numer of distance calculated during merge: {distance_count}') # 588321997

Numer of distance calculated during merge: 588321997


In [21]:
_, test_data, groundtruth_data = load_sift_dataset(train_file = None,
                                                      test_file='../datasets/sift1m-128d/sift_query.fvecs',
                                                      groundtruth_file='../datasets/sift1m-128d/sift_groundtruth.ivecs')

In [32]:
search_ef = 32
distance_count = 0
recall, _ = calculate_recall(hnsw_merged2, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:16<00:00, 131.02it/s]

32,0.8310200000000001, 610.77


In [33]:
search_ef = 40
distance_count = 0
recall, _ = calculate_recall(hnsw_merged2, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:36<00:00, 103.25it/s]

40,0.8628600000000001, 706.6628


In [34]:
search_ef = 64
distance_count = 0
recall, _ = calculate_recall(hnsw_merged2, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:54<00:00, 57.23it/s]

64,0.9152800000000001, 986.3936
